In [3]:
from my_weapon import *
from SQLite_handler import *
import tqdm as tqdm
from collections import defaultdict
plt.style.use('fivethirtyeight')

USA_STATES = ['NY', 'DC', 'IN', 'AR', 'WY', 'ME', 'TX', 'NH', 'CO', 'CA', 'IL',
             'WA', 'VA', 'FL', 'MA', 'OR', 'AZ', 'MT', 'MN', 'NE', 'TN', 'OH',
             'NJ', 'NV', 'KY', 'UT', 'NC', 'SC', 'PA', 'NM', 'KS', 'GA', 'MI',
             'WI', 'AK', 'MS', 'MD', 'LA', 'HI', 'MO', 'AL', 'CT', 'OK', 'IA',
             'WV', 'RI', 'SD', 'VT', 'ND', 'ID', 'DE']

In [16]:
#-->Matteo 2020/10/28
##Merge all the users we got soo far.
old_data=pd.read_csv('/home/alex/kayzhou/US_election/disk/users-location/202006-all.csv')
users_id_zk=set(old_data['uid'])

/home/alex/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
#Here i will just add those users of wich we have the county information.
#In data/county/County_users.lj contain all the users with county from Junuary to October
c=0
d=pd.DataFrame(columns=['uid','county','loc','name','state'])
new_ones=[]
for i in open('data/county_users/County_users.lj'):
    user=json.loads(i)
    #this because I'm stupid
    if isinstance(user, str):
            user=json.loads(user)
    if user['id'] in users_id_zk:
        continue
    new_ones.append([user['id'],user['loc'],None,None,user['loc'].split(',')[-1]])

In [40]:
d

,uid,county,loc,name,state


# Rural and Urban

In [ ]:
rural_table = pd.read_csv("data/ruralurbancodes2013.csv")
rural_table["Population"] = rural_table["Population_2010"].apply(lambda x: int(x.replace(",", "")))
rural_table["is_rural"] = rural_table["RUCC_2013"].apply(lambda x: 1 if x >= 3 else 0) # 分类，大于3以上是大城市
# rural_table[rural_table.County_Name=="Orleans Parish"]

# def county_to_rural(state_name, county_name):
#     try:
#         return rural_table[(rural_table["State"]==state_name) & (rural_table["county"]==county_name)]["is_rural"].iloc[0]
#     except:
#         return f"cannot find the loc ({state_name}, {county_name})"


dict_state_county_is_rural = defaultdict(dict)

for row in rural_table.itertuples():
    State = getattr(row, "State")
    county = getattr(row, "County_Name")
    is_rural = getattr(row, "is_rural")
    if county != "District of Columbia":
        county = " ".join(county.split()[: -1])
    dict_state_county_is_rural[State][county] = is_rural
    
# print(dict_state_county_is_rural["LA"])

rural_population = rural_table.groupby(["State", "is_rural"]).sum()["Population"]
rural_count = dict(rural_population)

In [ ]:
rural_table

## Load the users' opinions and locations

1. 累计预测模型结果（并没有细化到州）
2. 识别到地理位置的用户的观点（也可以看看全局结果）
3. 根据用户地理位置的经济状况，rescale后的结果

In [ ]:
# 载入用户观点

# load users' opinion
# users_opinion = pd.read_csv("data/tweetid_userid_pro_hillary.csv")
import pendulum
users_opinion = defaultdict(lambda: [0, 0])
# set_users = set(users_loc.index)
# start = pendulum.datetime(2016, 9, 1)

# election date: 2016-11-8
# for line in tqdm(open("data/tweetid_userid_pro_hillary.csv")):
#     w = line.strip().split(",")
#     uid = int(w[0])
#     if uid in set_users:
#         if float(w[2]) < 0.5:
#             users_opinion[uid][0] += 1
#         else:
#             users_opinion[uid][1] += 1

users_opinion = []
i = 0
for line in tqdm(open("data/cul01-2020-07-01.csv")):
    w = line.strip().split(",")
    i += 1
    if i == 1:
        continue
    uid = str(w[0])
    users_opinion.append({
        "uid": uid,
        "Camp": "T" if int(w[2]) > int(w[1]) else "B" 
    })
users_opinion = pd.DataFrame(users_opinion).set_index("uid")
users_opinion

In [ ]:
users_opinion["Camp"].value_counts(normalize=True)

In [ ]:
# 载入所有有地理位置信息的用户
users_loc = pd.read_csv("data/202006-all-users.csv", usecols=["uid", "state", "county"],
                        dtype={"uid": str, "state": str, "county": str},
                        index_col="uid")
users_loc = users_loc.dropna()
users_loc["county"] = users_loc["county"].apply(lambda x: " ".join(x.split(", ")[:-1]))
users_loc

3072442个用户拥有state和county信息。一定要有state数据，county可能不存在。

这些人的观点呢？

In [ ]:
users_opinion_with_location = users_loc.join(users_opinion)

In [ ]:
users_opinion_with_location

In [ ]:
users_opinion_with_location["Camp"].value_counts()

In [ ]:
users_opinion_with_location["Camp"].value_counts(normalize=True)

In [ ]:
# Since we only know the name of city, but the county. We need convert the city to county.
# check some mapping from http://www.statsamerica.org/CityCountyFinder/Default.aspx

# 需要更加详细的转换
_dict_city_to_county = {
    'Chicago, IL': 'Cook',
    'Washington, DC': 'District of Columbia',
    'Atlanta, GA': 'Fulton',
    'Boston, MA': 'Suffolk',
    'Seattle, WA': 'King',
    'Brooklyn, NY': 'New York',
    'Miami, FL': 'Miami-Dade',
    'Las Vegas, NV': 'Clark',
    'Portland, OR': 'Multnomah',
    'Pittsburgh, PA': 'Allegheny',
    'San Antonio, TX': 'Bexar',
    'Nashville, TN': 'Davidson',
    'Orlando, FL': 'Orange',
    'Phoenix, AZ': 'Maricopa',
    'Indianapolis, IN': 'Marion',
    'Columbus, OH': 'Franklin',
    'Minneapolis, MN': 'Hennepin',
    'St Louis, MO': 'St. Louis',
    'Charlotte, NC': 'Mecklenburg',
    'Cleveland, OH': 'Cuyahoga',
    'Detroit, MI': 'Wayne',
    'Tampa, FL': 'Hillsborough',
    'New Orleans, LA': 'Orleans',
    'Kansas City, MO': 'Jackson',
    'Cincinnati, OH': 'Hamilton'
}
    
def city_to_county(state_name, city_name):
    return _dict_city_to_county.get(f"{city_name}, {state_name}", city_name)

In [ ]:
from collections import Counter

find_county = Counter()
cannot_find_county = Counter()

for _state in tqdm(USA_STATES):
    _users = users_loc[users_loc.state==_state].copy()
    _users["county"] = _users["county"].apply(lambda x: city_to_county(_state, x))
    _users["is_rural"] = _users["county"].apply(lambda x: dict_state_county_is_rural[_state].get(x, -1))
    _users.to_csv(f"data/is_rural/{_state}.csv")

#     for row in _users.itertuples():
#         county = getattr(row, "county")
#         is_rural = getattr(row, "is_rural")
#         if is_rural == -1:
#             cannot_find_county[f"{county}, {_state}"] += 1
#         else:
#             find_county[f"{county}, {_state}"] += 1
    
    
# -1 represents that we cannot map the city (from BingMap) to county
# So we need more infomation between city and county

In [ ]:
print(len(cannot_find_county))
print(cannot_find_county.most_common(10))
find_county.most_common(10)

从1月~6月，共12988437个用户

In [ ]:
for _state in tqdm(USA_STATES):
    users_county = pd.read_csv("data/is_rural/" + _state + ".csv", index_col="uid")
    users_county = users_county.join(users_opinion)
    groups = dict(users_county.groupby(["is_rural", "Camp"]).size())
    # print(_state, groups) 

    w1 = rural_count.get((_state, 1), 0) / (rural_count.get((_state, 1), 0) + rural_count.get((_state, 0), 0))
    w0 = 1 - w1
    
    groups_T_H = dict(users_county[users_county.is_rural != -1].groupby("Camp").size())
    # print(groups_T_H)
    proT = groups_T_H.get("T", 0) / (groups_T_H.get("T", 0) + groups_T_H.get("B", 0))
    

    if (0, 'T') not in groups: # No urban people support Trump!
        pro_T_1 = groups[(1, 'T')] / (groups[(1, 'T')] + groups[(1, 'B')])
        rst = pro_T_1
    elif (1, 'T') not in groups: # No rural people support Trump!
        pro_T_0 = groups[(0, 'T')] / (groups[(0, 'T')] + groups[(0, 'B')])
        rst = pro_T_0
    else:
        pro_T_0 = groups[(0, 'T')] / (groups[(0, 'T')] + groups[(0, 'B')])
        pro_T_1 = groups[(1, 'T')] / (groups[(1, 'T')] + groups[(1, 'B')])
        rst = pro_T_1 * w1 + pro_T_0 * w0
        
    print(f"{_state}, Census (rural areas): {w1 * 100:.2f}%, Trump: {proT * 100:.2f}%, After rescaling: {rst * 100:.2f}%")

In [ ]:
users_counties = users_loc.join(users_opinion)

In [ ]:
users_counties

## 更细致的county转换

In [ ]:
CityCounty = pd.read_csv("data/Borror_CityCounty.csv")

In [ ]:
CityCounty

In [ ]:
CityCounty.columns